# WRC Scraper  2020

In [1]:
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize

In [2]:
# Cache results in text notebook
import requests_cache
requests_cache.install_cache('wrc_cache',
                             backend='sqlite',
                             expire_after=300)

In [91]:
s = requests.Session()
s.get('https://www.wrc.com')

<Response [200]>

In [4]:
# TO DO 
# There is also an: activeSeasonId":19
# IS there something we can get there?

In [5]:
#Is this URL constant or picked up relative to each rally?
URL='https://www.wrc.com/ajax.php?contelPageId=176146'

In [6]:
def _getActiveRally(_url=None):
    """Get active rally details."""
    if not _url:
        _url = 'https://www.wrc.com/ajax.php?contelPageId=171091'
    
    args= {"command":"getActiveRally","context":None}
    r = s.post(_url, data=json.dumps(args))
    event = json_normalize(r.json()).drop(columns='eventDays')
    days =  json_normalize(r.json(), 'eventDays').drop(columns='spottChannel.assets')
    channels = json_normalize(r.json(), ['eventDays', 'spottChannel','assets'])
    return (event, days, channels)

def getActiveRallyBase():
    """Get active rally using alt_url """
    event, days, channels = _getActiveRally()
    return (event, days, channels)

In [7]:
event, days, channels = getActiveRallyBase()
display(event.head())
display(days.head())
display(channels.head())
display(event.columns)

,id,name,externalIdRally,externalIdEvent,timezone,active,countdown,jwrc,winner,images.format16x9.320x180,...,status.id,status.name,pageInfo.id,pageInfo.title,pageInfo.feTitle,pageInfo.url,pageResult.id,pageResult.title,pageResult.feTitle,pageResult.url
0,100,Rallye Monte Carlo,153,124,1,True,False,False,None,https://www.wrc.com/images/redaktion/Web-2020/...,...,21,Live Event,38227,Rallye-Monte-Carlo,Rallye Monte-Carlo,/en/championship/calendar/wrc/rallye-monte-car...,89623,Rally Monte Carlo,Rally Monte Carlo,/en/results/rally-results/rally-monte-carlo/re...


,id,eventDay,spottChannel.id,spottChannel.displayName
0,331,2020-01-22,2,WRC Rallye Monte Carlo
1,334,2020-01-23,2,WRC Rallye Monte Carlo
2,341,2020-01-24,2,WRC Rallye Monte Carlo
3,344,2020-01-25,2,WRC Rallye Monte Carlo
4,355,2020-01-26,2,WRC Rallye Monte Carlo


,id,start,startUnix,end,endUnix,duration,alternative.title,alternative.description,alternative.image.480x270,alternative.image.thumbnail,...,content.image.800x450,content.image.thumbnail,content.image.original,content.dateTime.date,content.dateTime.timezone_type,content.dateTime.timezone,content.payment.id,content.payment.name,content.status.id,content.status.name
0,227,2020-01-22T15:00:00+00:00,1579705200,2020-01-22T15:40:00+00:00,1579707600,1548,Preview Magazine,None,https://ott.wrc.com/image/480/270/5e1e0da6bd29...,https://ott.wrc.com/image/thumbnail/5e1e0da6bd...,...,https://ott.wrc.com/image/800/450/5e21bfd8e014...,https://ott.wrc.com/image/thumbnail/5e21bfd8e0...,https://ott.wrc.com/image/original/5e21bfd8e01...,2020-01-17 14:30:00.000000,1,+00:00,3,Pay,3,Live
1,64,2020-01-23T18:00:00+00:00,1579802400,2020-01-23T18:30:00+00:00,1579804200,0,Good Evening Rally Fans - Service Gap,None,https://ott.wrc.com/image/480/270/5e2618150992...,https://ott.wrc.com/image/thumbnail/5e26181509...,...,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,https://ott.wrc.com/image/original/placeholder...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live
2,65,2020-01-23T18:30:00+00:00,1579804200,2020-01-23T19:15:00+00:00,1579806900,0,eSPORTS (TV Live),None,https://ott.wrc.com/image/480/270/5e1e100fce9e...,https://ott.wrc.com/image/thumbnail/5e1e100fce...,...,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,https://ott.wrc.com/image/original/placeholder...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live
3,239,2020-01-23T19:15:00+00:00,1579806900,2020-01-23T19:30:00+00:00,1579807800,0,Break,None,https://ott.wrc.com/image/480/270/5e25b39e1be1...,https://ott.wrc.com/image/thumbnail/5e25b39e1b...,...,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,https://ott.wrc.com/image/original/placeholder...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live
4,68,2020-01-23T19:30:00+00:00,1579807800,2020-01-23T20:30:00+00:00,1579811400,0,SS1 Malijai - Puimichel (TV LIVE),None,https://ott.wrc.com/image/480/270/5e1dfc31d968...,https://ott.wrc.com/image/thumbnail/5e1dfc31d9...,...,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,https://ott.wrc.com/image/original/placeholder...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live


Index(['id', 'name', 'externalIdRally', 'externalIdEvent', 'timezone',
       'active', 'countdown', 'jwrc', 'winner', 'images.format16x9.320x180',
       'images.format16x9.160x90', 'images.format16x9.path', 'season.id',
       'season.year', 'season.externalId', 'season.active', 'rally.id',
       'rally.name', 'rally.nation.id', 'rally.nation.name',
       'rally.nation.isoNumCode', 'rally.nation.alpha2', 'rally.nation.alpha3',
       'rally.nation.ioc', 'status.id', 'status.name', 'pageInfo.id',
       'pageInfo.title', 'pageInfo.feTitle', 'pageInfo.url', 'pageResult.id',
       'pageResult.title', 'pageResult.feTitle', 'pageResult.url'],
      dtype='object')

In [8]:
#Raw https://webappsdata.wrc.com/srv API?
#Need to create separate package to query that API
#Season info
#_url = 'https://webappsdata.wrc.com/srv/wrc/json/api/wrcsrv/byType?t=%22Season%22&maxdepth=1' 
#r = s.get(_url)
#json_normalize(r.json())

In [9]:
def getCurrentSeasonEvents():
    """Get events for current season"""
    _url='https://www.wrc.com/ajax.php?contelPageId=181782'
    #There seems to be a second UTL giving same data?
    #_url='https://www.wrc.com/ajax.php?contelPageId=183400'
    args = {"command":"getActiveSeason","context":None}
    r = s.post(_url, data=json.dumps(args))
    current_season_events = json_normalize(r.json(), ['rallyEvents', 'items'], meta='seasonYear').drop(columns='eventDays')
    eventdays = json_normalize(r.json(), ['rallyEvents', 'items', 'eventDays']).drop(columns='spottChannel.assets')
    eventchannel = json_normalize(r.json(), ['rallyEvents', 'items', 'eventDays', 'spottChannel','assets'])
    return (current_season_events, eventdays, eventchannel)


In [10]:
current_season_events, eventdays, eventchannel = getCurrentSeasonEvents()
display(current_season_events.head())
display(eventdays.head())
display(eventchannel.head())
display(current_season_events.columns)

,id,name,externalIdRally,externalIdEvent,timezone,active,countdown,jwrc,winner,images.format16x9.320x180,...,status.name,pageInfo.id,pageInfo.title,pageInfo.feTitle,pageInfo.url,pageResult.id,pageResult.title,pageResult.feTitle,pageResult.url,seasonYear
0,100,Rallye Monte Carlo,153,124,1,True,False,False,None,https://www.wrc.com/images/redaktion/Web-2020/...,...,Live Event,38227,Rallye-Monte-Carlo,Rallye Monte-Carlo,/en/championship/calendar/wrc/rallye-monte-car...,89623,Rally Monte Carlo,Rally Monte Carlo,/en/results/rally-results/rally-monte-carlo/re...,2020
1,102,Rally Sweden,154,125,2,False,False,True,None,https://www.wrc.com/images/redaktion/Web-2020/...,...,Pre Event,1914,Rally Sweden,Rally Sweden,/en/championship/calendar/wrc/rally-sweden/ove...,90029,Rally Sweden,Rally Sweden,/en/results/rally-results/rally-sweden/results/,2020
2,107,Rally Guanajuato Mexico,155,126,-6,False,False,False,None,https://www.wrc.com/images/redaktion/Web-2020/...,...,Pre Event,3083,Rally-Mexico,Rally México,/en/championship/calendar/wrc/rally-mexico/ove...,90044,Rally Mexico,Rally Mexico,/en/results/rally-results/rally-mexico/results/,2020
3,114,Rally Argentina,156,127,-3,False,False,False,None,https://www.wrc.com/images/redaktion/Web-2020/...,...,Pre Event,51444,Rally-Argentina,Rally Argentina,/en/championship/calendar/wrc/rally-argentina/...,90072,Rally Argentina,Rally Argentina,/en/results/rally-results/rally-argentina/resu...,2020
4,116,Rally de Portugal,157,128,1,False,False,False,None,https://www.wrc.com/images/redaktion/Web-2020/...,...,Pre Event,85219,Rally Portugal,Rally de Portugal,/en/championship/calendar/wrc/rally-de-portuga...,90091,Rally Portugal,Rally Portugal,/en/results/rally-results/rally-portugal/results/,2020


,id,eventDay,spottChannel.id,spottChannel.displayName
0,331,2020-01-22,2,WRC Rallye Monte Carlo
1,334,2020-01-23,2,WRC Rallye Monte Carlo
2,341,2020-01-24,2,WRC Rallye Monte Carlo
3,344,2020-01-25,2,WRC Rallye Monte Carlo
4,355,2020-01-26,2,WRC Rallye Monte Carlo


,id,start,startUnix,end,endUnix,duration,alternative.title,alternative.description,alternative.image.480x270,alternative.image.thumbnail,...,content.image.800x450,content.image.thumbnail,content.image.original,content.dateTime.date,content.dateTime.timezone_type,content.dateTime.timezone,content.payment.id,content.payment.name,content.status.id,content.status.name
0,227,2020-01-22T15:00:00+00:00,1579705200,2020-01-22T15:40:00+00:00,1579707600,1548,Preview Magazine,None,https://ott.wrc.com/image/480/270/5e1e0da6bd29...,https://ott.wrc.com/image/thumbnail/5e1e0da6bd...,...,https://ott.wrc.com/image/800/450/5e21bfd8e014...,https://ott.wrc.com/image/thumbnail/5e21bfd8e0...,https://ott.wrc.com/image/original/5e21bfd8e01...,2020-01-17 14:30:00.000000,1,+00:00,3,Pay,3,Live
1,64,2020-01-23T18:00:00+00:00,1579802400,2020-01-23T18:30:00+00:00,1579804200,0,Good Evening Rally Fans - Service Gap,None,https://ott.wrc.com/image/480/270/5e2618150992...,https://ott.wrc.com/image/thumbnail/5e26181509...,...,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,https://ott.wrc.com/image/original/placeholder...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live
2,65,2020-01-23T18:30:00+00:00,1579804200,2020-01-23T19:15:00+00:00,1579806900,0,eSPORTS (TV Live),None,https://ott.wrc.com/image/480/270/5e1e100fce9e...,https://ott.wrc.com/image/thumbnail/5e1e100fce...,...,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,https://ott.wrc.com/image/original/placeholder...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live
3,239,2020-01-23T19:15:00+00:00,1579806900,2020-01-23T19:30:00+00:00,1579807800,0,Break,None,https://ott.wrc.com/image/480/270/5e25b39e1be1...,https://ott.wrc.com/image/thumbnail/5e25b39e1b...,...,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,https://ott.wrc.com/image/original/placeholder...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live
4,68,2020-01-23T19:30:00+00:00,1579807800,2020-01-23T20:30:00+00:00,1579811400,0,SS1 Malijai - Puimichel (TV LIVE),None,https://ott.wrc.com/image/480/270/5e1dfc31d968...,https://ott.wrc.com/image/thumbnail/5e1dfc31d9...,...,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,https://ott.wrc.com/image/original/placeholder...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live


Index(['id', 'name', 'externalIdRally', 'externalIdEvent', 'timezone',
       'active', 'countdown', 'jwrc', 'winner', 'images.format16x9.320x180',
       'images.format16x9.160x90', 'images.format16x9.path', 'season.id',
       'season.year', 'season.externalId', 'season.active', 'rally.id',
       'rally.name', 'rally.nation.id', 'rally.nation.name',
       'rally.nation.isoNumCode', 'rally.nation.alpha2', 'rally.nation.alpha3',
       'rally.nation.ioc', 'status.id', 'status.name', 'pageInfo.id',
       'pageInfo.title', 'pageInfo.feTitle', 'pageInfo.url', 'pageResult.id',
       'pageResult.title', 'pageResult.feTitle', 'pageResult.url',
       'seasonYear'],
      dtype='object')

## getItinerary

In [11]:
def getActiveRally():
    """Get active rally details."""
    event, days, channels = _getActiveRally(URL)
    return (event, days, channels)

In [12]:
event, days, channels = getActiveRally()
display(event)
display(days)
display(channels.head())

,id,name,externalIdRally,externalIdEvent,timezone,active,countdown,jwrc,winner,images.format16x9.320x180,...,status.id,status.name,pageInfo.id,pageInfo.title,pageInfo.feTitle,pageInfo.url,pageResult.id,pageResult.title,pageResult.feTitle,pageResult.url
0,100,Rallye Monte Carlo,153,124,1,True,False,False,None,https://www.wrc.com/images/redaktion/Web-2020/...,...,21,Live Event,38227,Rallye-Monte-Carlo,Rallye Monte-Carlo,/en/championship/calendar/wrc/rallye-monte-car...,89623,Rally Monte Carlo,Rally Monte Carlo,/en/results/rally-results/rally-monte-carlo/re...


,id,eventDay,spottChannel.id,spottChannel.displayName
0,331,2020-01-22,2,WRC Rallye Monte Carlo
1,334,2020-01-23,2,WRC Rallye Monte Carlo
2,341,2020-01-24,2,WRC Rallye Monte Carlo
3,344,2020-01-25,2,WRC Rallye Monte Carlo
4,355,2020-01-26,2,WRC Rallye Monte Carlo


,id,start,startUnix,end,endUnix,duration,alternative.title,alternative.description,alternative.image.480x270,alternative.image.thumbnail,...,content.image.800x450,content.image.thumbnail,content.image.original,content.dateTime.date,content.dateTime.timezone_type,content.dateTime.timezone,content.payment.id,content.payment.name,content.status.id,content.status.name
0,227,2020-01-22T15:00:00+00:00,1579705200,2020-01-22T15:40:00+00:00,1579707600,1548,Preview Magazine,None,https://ott.wrc.com/image/480/270/5e1e0da6bd29...,https://ott.wrc.com/image/thumbnail/5e1e0da6bd...,...,https://ott.wrc.com/image/800/450/5e21bfd8e014...,https://ott.wrc.com/image/thumbnail/5e21bfd8e0...,https://ott.wrc.com/image/original/5e21bfd8e01...,2020-01-17 14:30:00.000000,1,+00:00,3,Pay,3,Live
1,64,2020-01-23T18:00:00+00:00,1579802400,2020-01-23T18:30:00+00:00,1579804200,0,Good Evening Rally Fans - Service Gap,None,https://ott.wrc.com/image/480/270/5e2618150992...,https://ott.wrc.com/image/thumbnail/5e26181509...,...,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,https://ott.wrc.com/image/original/placeholder...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live
2,65,2020-01-23T18:30:00+00:00,1579804200,2020-01-23T19:15:00+00:00,1579806900,0,eSPORTS (TV Live),None,https://ott.wrc.com/image/480/270/5e1e100fce9e...,https://ott.wrc.com/image/thumbnail/5e1e100fce...,...,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,https://ott.wrc.com/image/original/placeholder...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live
3,239,2020-01-23T19:15:00+00:00,1579806900,2020-01-23T19:30:00+00:00,1579807800,0,Break,None,https://ott.wrc.com/image/480/270/5e25b39e1be1...,https://ott.wrc.com/image/thumbnail/5e25b39e1b...,...,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,https://ott.wrc.com/image/original/placeholder...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live
4,68,2020-01-23T19:30:00+00:00,1579807800,2020-01-23T20:30:00+00:00,1579811400,0,SS1 Malijai - Puimichel (TV LIVE),None,https://ott.wrc.com/image/480/270/5e1dfc31d968...,https://ott.wrc.com/image/thumbnail/5e1dfc31d9...,...,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,https://ott.wrc.com/image/original/placeholder...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live


In [13]:
#This seems to work with sdbRallyId=None, returning active rally?
def getItinerary(sdbRallyId=None):
    """Get itinerary details for specified rally."""
    args = {"command":"getItinerary",
            "context":{"sdbRallyId":sdbRallyId}}
    r = s.post(URL, data=json.dumps(args))
    
    if not r.text or r.text=='null':
        itinerary = legs = sections = controls = stages = None
        return (itinerary, legs, sections, controls, stages)
    
    itinerary = json_normalize(r.json()).drop(columns='itineraryLegs')
    legs = json_normalize(r.json(),'itineraryLegs' )
    if not legs.empty:
        legs = legs.drop(columns='itinerarySections')
        sections = json_normalize(r.json(),['itineraryLegs', 'itinerarySections'] ).drop(columns=['controls','stages'])
        controls = json_normalize(r.json(),['itineraryLegs', 'itinerarySections', 'controls' ] )
        stages = json_normalize(r.json(),['itineraryLegs', 'itinerarySections', 'stages' ] )
    else:
        legs = sections = controls = stages = None
    return (itinerary, legs, sections, controls, stages)

In [14]:
sdbRallyId = 100
itinerary, legs, sections, controls, stages = getItinerary(sdbRallyId)
display(itinerary.head())
display(legs.head())
display(sections.head())
display(controls.head())
display(stages.head())

,itineraryId,eventId,name,priority
0,240,124,Itinerary,1


,itineraryLegId,itineraryId,startListId,name,legDate,order,status
0,273,240,451,Thursday 23rd January,2020-01-23,1,Completed
1,272,240,452,Friday 24th January,2020-01-24,2,Completed
2,275,240,454,Saturday 25th January,2020-01-25,3,Completed
3,274,240,456,Sunday 26th January,2020-01-25,4,Completed


,itinerarySectionId,itineraryLegId,order,name
0,637,273,1,Section 1
1,638,272,2,Section 2
2,639,272,3,Section 3
3,640,275,4,Section 4
4,641,275,5,Section 5


,controlId,eventId,stageId,type,code,location,timingPrecision,distance,targetDuration,targetDurationMs,firstCarDueDateTime,firstCarDueDateTimeLocal,status,controlPenalties,roundingPolicy,locked
0,6539,124,NaN,TimeControl,TC0,Monaco,Minute,0.00,None,NaN,2020-01-23T16:00:00,2020-01-23T17:00:00+01:00,Completed,All,NoRounding,True
1,6543,124,NaN,TimeControl,TC0A,Tyre Fitting Zone IN,Minute,166.33,02:45:00,9900000.0,2020-01-23T18:45:00,2020-01-23T19:45:00+01:00,Completed,All,NoRounding,True
2,6541,124,NaN,TimeControl,TC0B,Tyre Fitting Zone OUT,Minute,0.35,00:15:00,900000.0,2020-01-23T19:00:00,2020-01-23T20:00:00+01:00,Completed,All,NoRounding,True
3,6593,124,1528.0,TimeControl,TC1,Malijai,Minute,17.08,00:35:00,2100000.0,2020-01-23T19:35:00,2020-01-23T20:35:00+01:00,Completed,All,NoRounding,True
4,6592,124,1528.0,StageStart,SS1,Malijai - Puimichel (Live TV),Minute,17.47,00:03:00,180000.0,2020-01-23T19:38:00,2020-01-23T20:38:00+01:00,Interrupted,None,RoundToClosestMinute,True


,stageId,eventId,number,name,distance,status,stageType,timingPrecision,locked,code
0,1528,124,1,Malijai - Puimichel (Live TV),17.47,Interrupted,SpecialStage,Tenth,True,SS1
1,1538,124,2,Bayons - Bréziers,25.49,Completed,SpecialStage,Tenth,True,SS2
2,1533,124,3,Curbans - Venterol 1,20.02,Completed,SpecialStage,Tenth,True,SS3
3,1534,124,4,Saint-Clément - Freissinières 1,20.68,Completed,SpecialStage,Tenth,True,SS4
4,1535,124,5,Avançon - Notre-Dame-du-Laus 1,20.59,Completed,SpecialStage,Tenth,True,SS5


In [15]:
def getStartlist(startListId):
    """Get a startlist given startlist ID."""
    args={'command': 'getStartlist',
          'context': {'activeItineraryLeg': { 'startListId': startListId} }}
    r = s.post(URL, data=json.dumps(args))
    
    if not r.text or r.text=='null':
        startList = startListItems = None
        return (startList,startListItems)
    
    startList = json_normalize(r.json()).drop(columns='startListItems')
    startListItems = json_normalize(r.json(), 'startListItems')
    
    return (startList,startListItems)
    

In [16]:
startListId = 451
startList,startListItems = getStartlist(startListId)
display(startList.head())
display(startListItems.head())

,startListId,eventId,publishedStatus,name
0,451,124,Published,Thursday


,startListItemId,startListId,entryId,startDateTime,startDateTimeLocal,order
0,20891,451,20745,2020-01-23T17:37:00Z,2020-01-23T18:37:00+01:00,64
1,20892,451,20743,2020-01-23T17:36:00Z,2020-01-23T18:36:00+01:00,63
2,20893,451,20722,2020-01-23T17:35:00Z,2020-01-23T18:35:00+01:00,62
3,20894,451,20741,2020-01-23T17:34:00Z,2020-01-23T18:34:00+01:00,61
4,20895,451,20740,2020-01-23T17:33:00Z,2020-01-23T18:33:00+01:00,60


In [17]:
def getCars(sdbRallyId):
    """Get cars for a specified rally."""
    args = {"command":"getCars","context":{"sdbRallyId":100}}
    r = s.post(URL, data=json.dumps(args))
    if not r.text or r.text=='null':
        cars = classes = None
        return (cars, classes)
    
    cars = json_normalize(r.json()).drop(columns='eventClasses')
    classes = json_normalize(r.json(), 'eventClasses')
    return (cars, classes)

In [18]:
cars, classes = getCars(sdbRallyId)
display(cars.head())
display(classes.head())
cars.head().columns

,tag,entryId,eventId,driverId,codriverId,manufacturerId,entrantId,groupId,tagId,entryListOrder,...,manufacturer.manufacturerId,manufacturer.name,manufacturer.logoFilename,entrant.entrantId,entrant.name,entrant.logoFilename,group.groupId,group.name,tag.tagId,tag.name
0,NaN,20683,124,524,525,33,166,10,NaN,1,...,33,Hyundai,hyundai,166,HYUNDAI SHELL MOBIS WORLD RALLY TEAM,None,10,WRC,NaN,NaN
1,NaN,20684,124,762,4883,33,166,10,NaN,2,...,33,Hyundai,hyundai,166,HYUNDAI SHELL MOBIS WORLD RALLY TEAM,None,10,WRC,NaN,NaN
2,NaN,20685,124,670,3027,84,91,10,NaN,3,...,84,Toyota,toyota,91,TOYOTA GAZOO RACING WRT,None,10,WRC,NaN,NaN
3,NaN,20686,124,534,553,84,91,10,NaN,4,...,84,Toyota,toyota,91,TOYOTA GAZOO RACING WRT,None,10,WRC,NaN,NaN
4,NaN,20687,124,566,2470,26,94,10,NaN,5,...,26,Ford,ford,94,M-SPORT FORD WORLD RALLY TEAM,None,10,WRC,NaN,NaN


,eventClassId,eventId,name
0,582,124,RC1
1,582,124,RC1
2,582,124,RC1
3,582,124,RC1
4,582,124,RC1


Index(['tag', 'entryId', 'eventId', 'driverId', 'codriverId', 'manufacturerId',
       'entrantId', 'groupId', 'tagId', 'entryListOrder', 'identifier',
       'vehicleModel', 'eligibility', 'priority', 'status', 'tyreManufacturer',
       'driver.personId', 'driver.countryId', 'driver.country.countryId',
       'driver.country.name', 'driver.country.iso2', 'driver.country.iso3',
       'driver.firstName', 'driver.lastName', 'driver.abbvName',
       'driver.fullName', 'driver.code', 'codriver.personId',
       'codriver.countryId', 'codriver.country.countryId',
       'codriver.country.name', 'codriver.country.iso2',
       'codriver.country.iso3', 'codriver.firstName', 'codriver.lastName',
       'codriver.abbvName', 'codriver.fullName', 'codriver.code',
       'manufacturer.manufacturerId', 'manufacturer.name',
       'manufacturer.logoFilename', 'entrant.entrantId', 'entrant.name',
       'entrant.logoFilename', 'group.groupId', 'group.name', 'tag.tagId',
       'tag.name'],
      d

In [19]:
def getRally(sdbRallyId):
    """Get rally details for specified rally."""
    args = {"command":"getRally","context":{"sdbRallyId":sdbRallyId}}
    r = s.post(URL, data=json.dumps(args))
    rally = json_normalize(r.json()).drop(columns=['eligibilities','groups'])
    eligibilities = json_normalize(r.json(),'eligibilities')
    groups = json_normalize(r.json(),'groups')
    return (rally, eligibilities, groups)

In [20]:
rally, eligibilities, groups = getRally(sdbRallyId)
display(rally.head())
display(eligibilities.head())
display(groups.head())

,rallyId,eventId,itineraryId,name,isMain,eventClasses
0,153,124,240,WRC,True,None


,0
0,M
1,None
2,WRC2
3,WRC3
4,RGT


,groupId,name
0,10,WRC
1,98,RALLY2
2,9,RGT
3,99,RALLY4
4,6,R3


In [21]:
def getOverall(sdbRallyId, stageId):
    """Get overall standings for specificed rally and stage."""
    args = {"command":"getOverall","context":{"sdbRallyId":sdbRallyId,
                                              "activeStage":{"stageId":stageId}}}
    r = s.post(URL, data=json.dumps(args))
    overall = json_normalize(r.json())
    return overall

In [22]:
stageId = 1528
overall = getOverall(sdbRallyId, stageId)
overall.head()

,entryId,stageTimeMs,stageTime,penaltyTimeMs,penaltyTime,totalTimeMs,totalTime,position,diffFirstMs,diffFirst,diffPrevMs,diffPrev
0,20685,593400,PT9M53.4S,0,PT0S,593400,PT9M53.4S,1,0,PT0S,0,PT0S
1,20683,595200,PT9M55.2S,0,PT0S,595200,PT9M55.2S,2,1800,PT1.8S,1800,PT1.8S
2,20686,595300,PT9M55.3S,0,PT0S,595300,PT9M55.3S,3,1900,PT1.9S,100,PT0.1S
3,20684,599800,PT9M59.8S,0,PT0S,599800,PT9M59.8S,4,6400,PT6.4S,4500,PT4.5S
4,20690,603600,PT10M3.6S,0,PT0S,603600,PT10M3.6S,5,10200,PT10.2S,3800,PT3.8S


In [23]:
def getSplitTimes(sdbRallyId,stageId):
    """Get split times for specified rally and stage."""
    args = {"command":"getSplitTimes",
            "context":{"sdbRallyId":sdbRallyId, "activeStage":{"stageId":stageId}}}
    r = s.post(URL, data=json.dumps(args))
    splitPoints = json_normalize(r.json(),'splitPoints')
    entrySplitPointTimes = json_normalize(r.json(), 'entrySplitPointTimes').drop(columns='splitPointTimes')
    splitPointTimes = json_normalize(r.json(), ['entrySplitPointTimes','splitPointTimes'])
    return (splitPoints, entrySplitPointTimes, splitPointTimes)

In [24]:
splitPoints, entrySplitPointTimes, splitPointTimes = getSplitTimes(sdbRallyId,stageId)
display(splitPoints.head())
display(entrySplitPointTimes.head())
display(splitPointTimes.head())

,splitPointId,stageId,number,distance
0,3089,1528,1,3.36
1,3090,1528,2,7.55
2,3091,1528,4,16.40
3,3094,1528,3,15.60


,entryId,startDateTime,startDateTimeLocal,stageTimeDurationMs,stageTimeDuration
0,20683,2020-01-23T19:38:00,2020-01-23T20:38:00+01:00,595200,00:09:55.2000000
1,20684,2020-01-23T19:41:00,2020-01-23T20:41:00+01:00,599800,00:09:59.8000000
2,20685,2020-01-23T19:44:00,2020-01-23T20:44:00+01:00,593400,00:09:53.4000000
3,20686,2020-01-23T19:47:00,2020-01-23T20:47:00+01:00,595300,00:09:55.3000000
4,20687,2020-01-23T19:50:00,2020-01-23T20:50:00+01:00,610500,00:10:10.5000000


,splitPointTimeId,splitPointId,entryId,elapsedDurationMs,elapsedDuration,splitDateTime,splitDateTimeLocal
0,101851,3089,20683,95400,PT1M35.4S,2020-01-23T19:39:35.4,2020-01-23T20:39:35.4+01:00
1,101852,3090,20683,227700,PT3M47.7S,2020-01-23T19:41:47.7,2020-01-23T20:41:47.7+01:00
2,101855,3094,20683,532000,PT8M52S,2020-01-23T19:46:52,2020-01-23T20:46:52+01:00
3,101857,3091,20683,554300,PT9M14.3S,2020-01-23T19:47:14.3,2020-01-23T20:47:14.3+01:00
4,101853,3089,20684,95500,PT1M35.5S,2020-01-23T19:42:35.5,2020-01-23T20:42:35.5+01:00


In [25]:
def getStageTimes(sdbRallyId,stageId):
    """Get stage times for specified rally and stage"""
    args = {"command":"getStageTimes",
            "context":{"sdbRallyId":sdbRallyId,
                       "activeStage":{"stageId":stageId}}}
    r = s.post(URL, data=json.dumps(args))
    stagetimes = json_normalize(r.json())
    return stagetimes

In [26]:
stagetimes = getStageTimes(sdbRallyId,stageId)
stagetimes.head()

,stageTimeId,stageId,entryId,elapsedDurationMs,elapsedDuration,status,source,position,diffFirstMs,diffFirst,diffPrevMs,diffPrev
0,85682,1528,20685,593400.0,00:09:53.4000000,Completed,Default,1.0,0.0,00:00:00,0.0,00:00:00
1,85717,1528,20683,595200.0,00:09:55.2000000,Completed,Default,2.0,1800.0,00:00:01.8000000,1800.0,00:00:01.8000000
2,85684,1528,20686,595300.0,00:09:55.3000000,Completed,Default,3.0,1900.0,00:00:01.9000000,100.0,00:00:00.1000000
3,85680,1528,20684,599800.0,00:09:59.8000000,Completed,Default,4.0,6400.0,00:00:06.4000000,4500.0,00:00:04.5000000
4,85712,1528,20690,603600.0,00:10:03.6000000,Completed,Default,5.0,10200.0,00:00:10.2000000,3800.0,00:00:03.8000000


In [27]:
def getStagewinners(sdbRallyId):
    """Get stage winners for specified rally."""
    args = {"command":"getStagewinners",
            "context":{"sdbRallyId":sdbRallyId}}
    r = s.post(URL, data=json.dumps(args))
    stagewinners = json_normalize(r.json())
    return stagewinners

In [28]:
stagewinners = getStagewinners(sdbRallyId)
stagewinners.head()

,stageId,entryId,stageName,elapsedDurationMs,elapsedDuration
0,1538,20684,Bayons - Bréziers,983700,00:16:23.7000000
1,1528,20685,Malijai - Puimichel (Live TV),593400,00:09:53.4000000
2,1533,20686,Curbans - Venterol 1,802000,00:13:22
3,1534,20686,Saint-Clément - Freissinières 1,703300,00:11:43.3000000
4,1535,20686,Avançon - Notre-Dame-du-Laus 1,780700,00:13:00.7000000


Should we return empty dataframes with appropriate columns, or `None`?

An advantage of returning an empty dataframe with labelled columns is that we can also use the column value list as a test of a returned column.

We need to be consistent so we can have a common, consistent way of dealing with empty responses. This means things like `is None` or `pd.DataFrame().empty` both have to be handled.

In [29]:
COLS_PENALTIES=['penaltyId','controlId','entryId','penaltyDurationMs','penaltyDuration','reason']
               
def getPenalties(sdbRallyId):
    """Get penalties for specified rally."""
    args = {"command":"getPenalties",
            "context":{"sdbRallyId":sdbRallyId}}
    r = s.post(URL, data=json.dumps(args))
    
    if not r.text:
        return pd.DataFrame(columns=COLS_PENALTIES)
    
    penalties = json_normalize(r.json())
    return penalties

In [30]:
penalties = getPenalties(sdbRallyId)
penalties.head()

,penaltyId,controlId,entryId,penaltyDurationMs,penaltyDuration,reason
0,725,6592,20730,10000,PT10S,FALSE START
1,726,6592,20753,10000,PT10S,FALSE START
2,727,6590,20760,10000,PT10S,1 MIN LATE
3,728,6590,20764,50000,PT50S,5 MINS LATE
4,729,6590,20769,10000,PT10S,1 MIN LATE


In [31]:
COLS_RETIREMENT = ['retirementId','controlId','entryId','reason','retirementDateTime','retirementDateTimeLocal','status']

def getRetirements(sdbRallyId):
    """Get retirements for specified rally."""
    args = {"command":"getRetirements",
            "context":{"sdbRallyId":sdbRallyId}}
    r = s.post(URL, data=json.dumps(args))
    
    if not r.text:
        return pd.DataFrame(columns=COLS_RETIREMENT)
    
    retirements = json_normalize(r.json())
    return retirements

In [32]:
retirements = getRetirements(sdbRallyId)
retirements.head()

,retirementId,controlId,entryId,reason,retirementDateTime,retirementDateTimeLocal,status
0,1475,6591,20710,OFF ROAD,2020-01-23T20:40:00Z,0001-01-01T00:00:00+00:00,Temporary
1,1476,6588,20687,MECHANICAL,2020-01-23T21:57:00Z,0001-01-01T00:00:00+00:00,Temporary
2,1477,6591,20750,OFF ROAD,2020-01-23T22:51:00Z,0001-01-01T00:00:00+00:00,Permanent
3,1478,6545,20687,REJOINED,2020-01-24T06:16:00Z,0001-01-01T00:00:00+00:00,Rejoined
4,1479,6545,20710,REJOINED,2020-01-24T06:17:00Z,0001-01-01T00:00:00+00:00,Rejoined


In [33]:
SEASON_URL = 'https://www.wrc.com/ajax.php?contelPageId=186641'


In [35]:
#How can we look these up?
SEASON_CATEGORIES = {'WRC':"35", "WRC2":"46", "WRC3":"49","JWRC":"58"}

In [36]:
def getSeasonCategory(seasonCategory=SEASON_CATEGORIES['WRC']): 
    """??"""
    args = {"command":"getSeasonCategory",
            "context":{"seasonCategory":seasonCategory}}
    r = s.post(SEASON_URL, data=json.dumps(args))
    if not r.text:
        return None
    return json_normalize(r.json())

In [37]:
getSeasonCategory()

,id,externalIdDriver,externalIdCoDriver,externalIdManufacturer,season.id,season.year,season.externalId,season.active,category.id,category.name,category.sorting
0,35,37,38,39,19,2020,6,True,7,WRC,1


In [38]:
SC_COLS = ['id','externalIdDriver','externalIdCoDriver','externalIdManufacturer']

def getChampionshipCodes():
    """Create dataframe of external championship IDs."""
    champs=pd.DataFrame()

    for sc in SEASON_CATEGORIES:
        seasonCategory = SEASON_CATEGORIES[sc]

        champs = champs.append(getSeasonCategory(seasonCategory)[SC_COLS])

    champs.set_index('id', inplace=True)
    champs.rename(columns={'externalIdDriver':'drivers',
                           'externalIdCoDriver':'codrivers',
                           'externalIdManufacturer':'manufacturers'},
                 inplace=True)
    return champs


In [39]:
getChampionshipCodes()

,drivers,codrivers,manufacturers
id,,,
35,37,38,39
46,40,41,43
49,44,45,None
58,46,47,None


In [149]:
class SeasonBase:
    """Base class for things to do with seasons."""
    def __init__(self, season_external_id=None, autoseed=False):
        self.season_external_id = season_external_id or None
        if not self.season_external_id and autoseed:
            self._check_season_external_id()
            
    def _check_season_external_id(self):
        """Check that season_external_id exists and if not, get one."""
        if not hasattr(self,'season_external_id') or not self.season_external_id:
            #Get current one from active rally
            #It's also available from current_season_events
            event, days, channels = getActiveRallyBase()
            self.event, self.days, self.channels = event, days, channels
            #The returned np.int64 is not JSON serialisable
            self.season_external_id = int(event.loc[0,'season.externalId'])

# TO DO
class Championship(SeasonBase):       
    """Class for championship."""
    def __init__(self ):
        SeasonBase.__init__(self)


In [150]:
SeasonBase(autoseed=True).season_external_id

6

In [151]:
def _getChampionshipId(category='WRC', typ='drivers'):
    """Look up external ids for championship by category and championship."""
    champs=getChampionshipCodes()
    championship_activeExternalId = champs.to_dict(orient='index')[int(SEASON_CATEGORIES[category])]
    activeExternalId = championship_activeExternalId[typ]
    return activeExternalId

def getChampionship(category='WRC',typ='drivers', season_external_id=None, ):
    """Get Championship details for specified category and championship.
       If nor season ID is provided, use the external seasonid from the active rally. """
    
    season_external_id = SeasonBase(autoseed=True).season_external_id
    args = {"command":"getChampionship",
            "context":{"season":{"externalId":season_external_id},
                       "activeExternalId":_getChampionshipId(category,typ)}}
    
    r = s.post(SEASON_URL, data=json.dumps(args))

    if not r.text:
        championship = championshipRounds = championshipEntries = None
        return (championship, championshipRounds, championshipEntries)
    
    championship = json_normalize(r.json()).drop(columns=['championshipRounds','championshipEntries'])
    championshipRounds = json_normalize(r.json(), 'championshipRounds' )
    championshipEntries = json_normalize(r.json(), 'championshipEntries')
    return (championship, championshipRounds, championshipEntries)

In [152]:
(championship, championshipRounds, championshipEntries) = getChampionship('JWRC', 'drivers')
display(championship)
display(championshipRounds.head())
display(championshipEntries.head())

6


,championshipId,seasonId,name,type,fieldOneDescription,fieldTwoDescription,fieldThreeDescription,fieldFourDescription,fieldFiveDescription
0,46,6,FIA Junior WRC Championship for Drivers,Drivers,FirstName,LastName,CountryISO3,Manufacturer,TyreManufacturer


,eventId,championshipId,order,event.eventId,event.countryId,event.country.countryId,event.country.name,event.country.iso2,event.country.iso3,event.name,...,event.timeZoneOffset,event.surfaces,event.organiserUrl,event.categories,event.mode,event.trackingEventId,event.clerkOfTheCourse,event.stewards,event.templateFilename,event.locked
0,125,46,1,125,215,215,Sweden,SE,SWE,Rally Sweden,...,60,,,None,Rally,3046,None,None,None,False
1,129,46,2,129,110,110,Italy,IT,ITA,Rally Italia Sardegna,...,60,,,None,Rally,3050,None,None,None,False
2,131,46,3,131,75,75,Finland,FI,FIN,Neste Rally Finland,...,120,,,None,Rally,3052,None,None,None,False
3,134,46,4,134,83,83,Germany,DE,DEU,ADAC Rallye Deutschland,...,60,,,None,Rally,3055,None,None,None,False
4,135,46,5,135,235,235,United Kingdom of Great Britain and Northern I...,GB,GBR,Dayinsure Rally Wales,...,0,,,None,Rally,3056,None,None,None,False


""


In [42]:
getChampionshipCodes().to_dict(orient='index')#[int(SEASON_CATEGORIES['JWRC'])]

{35: {'drivers': 37, 'codrivers': 38, 'manufacturers': 39},
 46: {'drivers': 40, 'codrivers': 41, 'manufacturers': 43},
 49: {'drivers': 44, 'codrivers': 45, 'manufacturers': None},
 58: {'drivers': 46, 'codrivers': 47, 'manufacturers': None}}

In [43]:
def getChampionshipStandings():
    """Get championship standings."""

    args = {"command":"getChampionshipStandings",
            "context":{
                       "season":{"externalId":6,
                                 },
                       "activeExternalId":"38"}}
    r = s.post(SEASON_URL, data=json.dumps(args))
    
    if not r.text:
        championship_standings = round_results = None
        return (championship_standings, round_results)
    championship_standings = json_normalize(r.json(),'entryResults', meta='championshipId').drop(columns='roundResults')
    round_results = json_normalize(r.json(),['entryResults', 'roundResults'])

    return (championship_standings, round_results)


In [44]:
championship_standings, round_results = getChampionshipStandings()
display(championship_standings.head())
display(round_results.head())

,championshipEntryId,overallPosition,overallPoints,championshipId
0,752,1,30,38
1,755,2,22,38
2,754,3,17,38
3,756,4,13,38
4,759,5,10,38


,championshipEntryId,championshipId,eventId,position,totalPoints,pointsBreakdown,dropped,status,publishedStatus
0,752,38,124,1,30,25 + 5,False,Finished,Published
1,755,38,124,2,22,18 + 4,False,Finished,Published
2,754,38,124,3,17,15 + 2,False,Finished,Published
3,756,38,124,4,13,12 + 1,False,Finished,Published
4,759,38,124,5,10,10,False,Finished,Published


In [83]:
# TO DO - define a class for each table
import warnings


class WRCRally_sdb:
    """Base class for things with an sdbRallyId.
       Can also help find an active sdbRallyId"""
    def __init__(self, sdbRallyId=None, live=False,
                 autoseed=False, nowarn=True,):
        if not nowarn and not sdbRallyId:
            warnings.warn("sdbRallyId should really be set...")
        
        self.sdbRallyId = sdbRallyId or None
        
        if autoseed:
            self._checkRallyId(sdbRallyId)
    
    def _checkRallyId(self, sdbRallyId=None):
        """Return a rally ID or lookup active one."""
        sdbRallyId = sdbRallyId or self.sdbRallyId
        if not hasattr(self, 'sdbRallyId') or self.itinerary
        if not sdbRallyId:
            self.activerally = WRCActiveRally()
            self.sdbRallyId = self.activerally.sdbRallyId
        return self.sdbRallyId

In [88]:
class WRCActiveRally(WRCRally_sdb):
    """Class for the active rally."""
    def __init__(self, live=False ):
        WRCRally_sdb.__init__(self, None, live, nowarn=True)

        self.live = live
        self.fetchData()
        
    def fetchData(self):
        event, days, channels = getActiveRally()
        self.event, self.days, self.channels = event, days, channels

        #np.int64 is not JSON serialisable
        self.sdbRallyId = int(event.loc[0,'id'])

        self.name = event.loc[0,'name']

In [89]:
WRCActiveRally()

In [84]:
zz = WRCRally_sdb(autoseed=True)
print(zz.sdbRallyId)

100


We use the `.fetchData()` method so as to ry not to be greedy. This way, we can define a class and start to work towards only grabbling the data if we need it.

In [103]:
class WRCRetirements(WRCRally_sdb):
    """Class for retirements"""
    def __init__(self, sdbRallyId=None, live=False, autoseed=False):
        """Initialise retirements class."""
        WRCRally_sdb.__init__(self, sdbRallyId, live, autoseed)
            
        self.retirements=None
        
        if self.sdbRallyId:
            self.fetchData(self.sdbRallyId)
        
    def fetchData(self, sdbRallyId=None):
        """Fetch the data from WRC API."""
        self._checkRallyId(sdbRallyId)
        self.retirements = getRetirements(self.sdbRallyId)
        

In [106]:
zz=WRCRetirements(autoseed=True)
zz.retirements.head(3)

,retirementId,controlId,entryId,reason,retirementDateTime,retirementDateTimeLocal,status
0,1475,6591,20710,OFF ROAD,2020-01-23T20:40:00Z,0001-01-01T00:00:00+00:00,Temporary
1,1476,6588,20687,MECHANICAL,2020-01-23T21:57:00Z,0001-01-01T00:00:00+00:00,Temporary
2,1477,6591,20750,OFF ROAD,2020-01-23T22:51:00Z,0001-01-01T00:00:00+00:00,Permanent


In [101]:
class WRCPenalties(WRCRally_sdb):
    """Class for penalties"""
    def __init__(self, sdbRallyId=None, live=False, autoseed=False):
        """Initialise penalties class."""
        WRCRally_sdb.__init__(self, sdbRallyId, live, autoseed)
            
        self.penalties=None
        
        if self.sdbRallyId:
            self.fetchData(self.sdbRallyId)
        
    def fetchData(self, sdbRallyId=None):
        """Fetch the data from WRC API."""
        self._checkRallyId(sdbRallyId)
        self.penalties = getPenalties(self.sdbRallyId)
        

In [107]:
zz=WRCPenalties(autoseed=True)
zz.penalties.head(3)

,penaltyId,controlId,entryId,penaltyDurationMs,penaltyDuration,reason
0,725,6592,20730,10000,PT10S,FALSE START
1,726,6592,20753,10000,PT10S,FALSE START
2,727,6590,20760,10000,PT10S,1 MIN LATE


In [60]:
rally, eligibilities, groups = getRally(sdbRallyId)
splitPoints, entrySplitPointTimes, splitPointTimes = getSplitTimes(sdbRallyId,stageId)
stagetimes = getStageTimes(sdbRallyId,stageId)
stagewinners = getStagewinners(sdbRallyId)
championship = getChampionship()
championship = getChampionshipStandingsLive()

NameError: name 'getChampionshipStandingsLive' is not defined

In [62]:
class WRCItinerary(WRCRally_sdb):
    """Class for WRC2020 Itinerary."""
    def __init__(self, sdbRallyId=None, live=False, autoseed=False):
        """Initialise itinerary class."""
        WRCRally_sdb.__init__(self, sdbRallyId, live, autoseed)
        
        self.itinerary=None
        self.legs=None
        self.sections=None
        self.controls=None
        self.stages=None
        
        if self.sdbRallyId:
            self.fetchData(sdbRallyId)
        
    def fetchData(self, sdbRallyId=None):
        """Fetch the data from WRC API."""
        self._checkRallyId(sdbRallyId)
        
        itinerary, legs, sections, controls, stages = getItinerary(sdbRallyId)
        self.itinerary, self.legs, self.sections, self.controls, self.stages = itinerary, legs, sections, controls, stages
        

In [70]:
print(WRCItinerary(autoseed=True).sdbRallyId)

None


In [63]:
WRCItinerary(sdbRallyId=100).legs

,itineraryLegId,itineraryId,startListId,name,legDate,order,status
0,273,240,451,Thursday 23rd January,2020-01-23,1,Completed
1,272,240,452,Friday 24th January,2020-01-24,2,Completed
2,275,240,454,Saturday 25th January,2020-01-25,3,Completed
3,274,240,456,Sunday 26th January,2020-01-25,4,Completed


In [108]:
class WRCStartlist():
    """Class for WRC2020 Startlist table."""
    def __init__(self, startlistId=None):
        self.startListId = startlistId or None
        
        if not self.startListId:
            warnings.warn("startListId should really be set..")
        
        if self.startListId:
            self.fetchData(startListId)
    
    def _checkStartListId(self, startListId=None):
        """Return a startlistId or look one up."""
        startListId = startListId or self.startListId
        if not startListId:
            if not hasattr(self, 'itinerary') or not self.itinerary:
                self.itinerary = WRCItinerary(autoseed=True)
                self.sdbRallyId = self.itinerary.sdbRallyId
            self.startListId = int(self.itinerary.legs.loc[0,'startListId'])
        return self.startListId
        
        
    def fetchData(self, startListId=None):
        self._checkStartListId(startListId)
        startList,startListItems = getStartlist(startListId)
        self.startList, self.startListItems = startList,startListItems

In [109]:
WRCStartlist().fetchData()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: startListId should really be set..
  import sys


KeyError: "['startListItems'] not found in axis"

In [749]:
class WRCCars(WRCRally_sdb):
    """Class for WRC2020 Cars table."""
    def __init__(self, sdbRallyId=None, live=False):  
        WRCRally_sdb.__init__(self, sdbRallyId, live)
        
        self.cars=None
        self.classes=None
        
        if self.sdbRallyId:
            self.fetchData(sdbRallyId)
            
    def fetchData(self, sdbRallyId=None):
        self._checkRallyId(sdbRallyId)
        cars, classes = getCars(sdbRallyId)
        self.cars, self.classes = cars, classes

In [750]:
WRCCars()

In [ ]:
#This class will contain everything about a single rally
class WRCRally(WRCRally_sbd):
    """Class for a rally - stuff where sdbRallyId is required."""
    def __init__(self, sdbRallyId=None, live=False, ):
        WRCRally_sbd.__init__(self, sdbRallyId, live)
        
        self.live = live
        self.itinerary = None
        self.startListId = None
        self.activerally = None
    
    
    def getItinerary(self):
        """Get itinerary.
           If rally not known, use active rally.
           Also set a default startListId."""
        
        _i = self.itinerary = WRCItinerary(self.sdbRallyId)
        
        #Set a default startListId value if required
        if not self.startListId and _i and _i.legs and not _i.legs.empty :
            self.startListId = int(_i.legs.loc[0,'startListId'])
            
        return (_i.itinerary, _i.legs, _i.sections, _i.controls, _i.stages)
 
    def getCars(self):
        """Get cars for a rally.
           If no rally provided, use current one."""
        
        _c = self.cars = WRCCars(self.sdbRallyId)
        
        return (_c.cars, _c.classes)
       
        
    def getStartlist(self, startListId=None):
        """Get startlist.
           If no startListId provided, try to find a default."""
        
        _s = self.startlist = WRCStartlist(self.startListId)
      
        return (_s.startList, _s.startListItems)
    
    
    
    

In [ ]:
# NEXT TO DO - active rally class

In [ ]:
zz = WRCRally()
zz.getStartlist()

In [ ]:
# TO DO - need a more gernal season events class?
# If, that is, we can we look up arbtrary season events...
class WRCCurrentSeasonEvents:
    """Class for Season events."""
    def __init__(self ):
        self.current_season_events, self.eventdays, self.eventchannel = getCurrentSeasonEvents()


In [755]:
#This class needs renaming...
#What does it actually represent? An event? A live event? A set of events?
class WRC2020(WRCRally):
    """Class for WRC data scrape using 2020 API."""

    def __init__(self, sdbRallyId=None, live=False):
        WRCRally.__init__(self, sdbRallyId, live)
        
        self.live = live
        self.currentseasonevents = None
        
        
    def getCurrentSeasonEvents(self):
        """Get Current season events."""
        if not self.currentseasonevents:
            _cse = self.currentseasonevents = WRCCurrentSeasonEvents()
        return (_cse.current_season_events, _cse.eventdays, _cse.eventchannel)


    
    
    
    

In [756]:
wrc=WRC2020()
wrc.getCurrentSeasonEvents()
wrc.currentseasonevents.current_season_events

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: sdbRallyId should really be set...
  if __name__ == '__main__':


,id,name,externalIdRally,externalIdEvent,timezone,active,countdown,jwrc,winner,images.format16x9.320x180,...,status.name,pageInfo.id,pageInfo.title,pageInfo.feTitle,pageInfo.url,pageResult.id,pageResult.title,pageResult.feTitle,pageResult.url,seasonYear
0,100,Rallye Monte Carlo,153,124,1,True,False,False,None,https://www.wrc.com/images/redaktion/Web-2020/...,...,Live Event,38227,Rallye-Monte-Carlo,Rallye Monte-Carlo,/en/championship/calendar/wrc/rallye-monte-car...,89623,Rally Monte Carlo,Rally Monte Carlo,/en/results/rally-results/rally-monte-carlo/re...,2020
1,102,Rally Sweden,154,125,2,False,False,True,None,https://www.wrc.com/images/redaktion/Web-2020/...,...,Pre Event,1914,Rally Sweden,Rally Sweden,/en/championship/calendar/wrc/rally-sweden/ove...,90029,Rally Sweden,Rally Sweden,/en/results/rally-results/rally-sweden/results/,2020
2,107,Rally Guanajuato Mexico,155,126,-6,False,False,False,None,https://www.wrc.com/images/redaktion/Web-2020/...,...,Pre Event,3083,Rally-Mexico,Rally México,/en/championship/calendar/wrc/rally-mexico/ove...,90044,Rally Mexico,Rally Mexico,/en/results/rally-results/rally-mexico/results/,2020
3,114,Rally Argentina,156,127,-3,False,False,False,None,https://www.wrc.com/images/redaktion/Web-2020/...,...,Pre Event,51444,Rally-Argentina,Rally Argentina,/en/championship/calendar/wrc/rally-argentina/...,90072,Rally Argentina,Rally Argentina,/en/results/rally-results/rally-argentina/resu...,2020
4,116,Rally de Portugal,157,128,1,False,False,False,None,https://www.wrc.com/images/redaktion/Web-2020/...,...,Pre Event,85219,Rally Portugal,Rally de Portugal,/en/championship/calendar/wrc/rally-de-portuga...,90091,Rally Portugal,Rally Portugal,/en/results/rally-results/rally-portugal/results/,2020
5,126,Rally Italia Sardegna,158,129,2,False,False,True,None,https://www.wrc.com/images/redaktion/Web-2020/...,...,Pre Event,85270,Rally Italia Sardegna,Rally Italia Sardegna,/en/championship/calendar/wrc/rally-italia-sar...,90113,Rally Italia Sardegna,Rally Italia Sardegna,/en/results/rally-results/rally-italia-sardegn...,2020
6,137,Safari Rally Kenya,159,130,3,False,False,False,None,https://www.wrc.com/images/redaktion/Web-2020/...,...,Pre Event,85348,Safari Rally Kenya,Rally Italia Sardegna,/en/championship/calendar/wrc/safari-rally-ken...,90134,Safari Rally Kenya,Safari Rally Kenya,/en/results/rally-results/safari-rally-kenya/r...,2020
7,148,Neste Rally Finland,160,131,3,False,False,True,None,https://www.wrc.com/images/redaktion/Web-2020/...,...,Pre Event,85394,Rally Finland,Neste Rally Finland,/en/championship/calendar/wrc/neste-rally-finl...,90150,Rally Finland,Rally Finland,/en/results/rally-results/rally-finland/results/,2020
8,157,Rally New Zealand,161,132,12,False,False,False,None,https://www.wrc.com/images/redaktion/Web-2020/...,...,Pre Event,85447,Rally New Zealand,Rally New Zealand,/en/championship/calendar/wrc/rally-new-zealan...,90174,Rally New Zealand,Rally New Zealand,/en/results/rally-results/rally-new-zealand/re...,2020
9,158,Rally Turkey,162,133,3,False,False,False,None,https://www.wrc.com/images/redaktion/Web-2020/...,...,Pre Event,85490,Rally Turkey,Rally Turkey,/en/championship/calendar/wrc/rally-turkey/ove...,90193,Rally Turkey,Rally Turkey,/en/results/rally-results/rally-turkey/results/,2020


In [456]:
wrc.getStartlist()

(   startListId  eventId publishedStatus      name
 0          451      124       Published  Thursday,
     startListItemId  startListId  entryId         startDateTime  \
 0             20891          451    20745  2020-01-23T17:37:00Z   
 1             20892          451    20743  2020-01-23T17:36:00Z   
 2             20893          451    20722  2020-01-23T17:35:00Z   
 3             20894          451    20741  2020-01-23T17:34:00Z   
 4             20895          451    20740  2020-01-23T17:33:00Z   
 ..              ...          ...      ...                   ...   
 83            20974          451    20702  2020-01-23T16:55:00Z   
 84            20975          451    20703  2020-01-23T16:54:00Z   
 85            20976          451    20707  2020-01-23T16:53:00Z   
 86            20977          451    20728  2020-01-23T17:15:00Z   
 87            20978          451    20683  2020-01-23T16:00:00Z   
 
            startDateTimeLocal  order  
 0   2020-01-23T18:37:00+01:00     64  

In [325]:
wrc.itinerary.sections

,itinerarySectionId,itineraryLegId,order,name
0,637,273,1,Section 1
1,638,272,2,Section 2
2,639,272,3,Section 3
3,640,275,4,Section 4
4,641,275,5,Section 5
5,642,274,6,Section 6
6,643,274,7,Section 7


In [280]:
itinerary, legs, sections, controls, stages = wrc.getItinerary()
startList,startListItems = getStartlist(startListId)
cars, classes = getCars(sdbRallyId)
#rally, eligibilities, groups = getRally(sdbRallyId)
#overall = getOverall(sdbRallyId, stageId)
#splitPoints, entrySplitPointTimes, splitPointTimes = getSplitTimes(sdbRallyId,stageId)
#stagetimes = getStageTimes(sdbRallyId,stageId)
#stagewinners = getStagewinners(sdbRallyId)
#penalties = getPenalties(sdbRallyId)
#retirements = getRetirements(sdbRallyId)
#championship = getChampionship()
#championship = getChampionshipStandingsLive()

In [236]:
current_season_events, eventdays, eventchannel = wrc.getCurrentSeasonEvents()

In [242]:
event, days, channels = wrc.getActiveRally()


In [238]:
event

,id,name,externalIdRally,externalIdEvent,timezone,active,countdown,jwrc,winner,images.format16x9.320x180,...,status.id,status.name,pageInfo.id,pageInfo.title,pageInfo.feTitle,pageInfo.url,pageResult.id,pageResult.title,pageResult.feTitle,pageResult.url
0,100,Rallye Monte Carlo,153,124,1,True,False,False,None,https://www.wrc.com/images/redaktion/Web-2020/...,...,21,Live Event,38227,Rallye-Monte-Carlo,Rallye Monte-Carlo,/en/championship/calendar/wrc/rallye-monte-car...,89623,Rally Monte Carlo,Rally Monte Carlo,/en/results/rally-results/rally-monte-carlo/re...


In [243]:
wrc.sdbRallyId

100